In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import base

In [3]:
from src.models import build_deep_mind_model
from src.agents import DeepQAgent

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
model = build_deep_mind_model()

In [5]:
DeepQAgent(model)

DeepQAgent(
    model=<keras.models.Sequential object at 0x7f586e9ba978>,
    learning_rate=0.001,
    discount_factor=0.99,
    exploration_rate=0.8,
    exploration_decay=0.99,
    episodes=1000
)